In [4]:
## Probamos otro modelo xgboost
!pip install xgboost


In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler

# Preprocessing 
from sklearn.preprocessing import MinMaxScaler , LabelEncoder , StandardScaler


#### Loadin data

In [18]:
data = pd.read_csv("../data/diamonds_train.csv")

In [19]:
data = data[['carat',
             'cut', 
             'color', 
             'clarity', 
             'depth', 
             'table', 
             'x', 
             'y', 
             'z', 
             'price']]
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,4268
1,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,505
2,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,2686
3,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,738
4,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,4882


In [20]:
data["size"] = data["x"] * data["y"] * data["z"]
data

,carat,cut,color,clarity,depth,table,x,y,z,price,size
0,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,4268,197.096725
1,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,505,52.395750
2,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,2686,113.436890
3,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,738,66.268800
4,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,4882,168.429975
...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,Ideal,G,VS1,62.7,57.0,7.10,7.04,4.43,10070,221.429120
40451,2.02,Good,F,SI2,57.1,60.0,8.31,8.25,4.73,12615,324.276975
40452,1.01,Ideal,H,SI1,62.7,56.0,6.37,6.42,4.01,5457,163.990554
40453,0.33,Ideal,J,VS1,61.9,54.3,4.45,4.47,2.76,456,54.900540


In [21]:
data = data[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'size']]
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  object 
 2   color    40455 non-null  object 
 3   clarity  40455 non-null  object 
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   price    40455 non-null  int64  
 7   size     40455 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 2.5+ MB


## Encode data

In [22]:
# Columnas categóricas
cat_col = ['cut', 'clarity', 'color']

# Inicializar LabelEncoder
le = LabelEncoder()

# Aplicar LabelEncoder a las columnas categóricas
for col in cat_col:
    data[col] = le.fit_transform(data[col])

# Información del DataFrame después de la codificación
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  int64  
 2   color    40455 non-null  int64  
 3   clarity  40455 non-null  int64  
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   price    40455 non-null  int64  
 7   size     40455 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 2.5 MB


/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## Train Test Split

In [23]:
X = data.drop('price', axis=1)
y = data['price'] 
# Your code here:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (32364, 7), X_test: (8091, 7), y_train: (32364,), y_test: (8091,)


In [84]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

In [85]:
xgb_model.fit(X, y)

/Users/sil/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:10:36] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [86]:
y_pred = xgb_model.predict(X)

In [87]:
mse=mean_squared_error(y, y_pred)

In [88]:
print(np.sqrt(mse))

409.28571651141414


## TEST

In [90]:
data = pd.read_csv("../data/diamonds_test.csv")

In [91]:
data = data[['carat',
             'cut', 
             'color', 
             'clarity', 
             'depth', 
             'table', 
             'x', 
             'y', 
             'z']]
data.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19


In [92]:
data["size"] = data["x"] * data["y"] * data["z"]
data

,carat,cut,color,clarity,depth,table,x,y,z,size
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,125.806866
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,196.129362
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,246.878712
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,145.593630
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,81.997355
...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,93.924600
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,116.477148
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,115.898725
13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,118.874925


In [93]:
data = data[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'size']]
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    13485 non-null  float64
 1   cut      13485 non-null  object 
 2   color    13485 non-null  object 
 3   clarity  13485 non-null  object 
 4   depth    13485 non-null  float64
 5   table    13485 non-null  float64
 6   size     13485 non-null  float64
dtypes: float64(4), object(3)
memory usage: 737.6+ KB


In [94]:
# Columnas categóricas
cat_col = ['cut', 'clarity', 'color']

# Inicializar LabelEncoder
le = LabelEncoder()

# Aplicar LabelEncoder a las columnas categóricas
for col in cat_col:
    data[col] = le.fit_transform(data[col])

# Información del DataFrame después de la codificación
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    13485 non-null  float64
 1   cut      13485 non-null  int64  
 2   color    13485 non-null  int64  
 3   clarity  13485 non-null  int64  
 4   depth    13485 non-null  float64
 5   table    13485 non-null  float64
 6   size     13485 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 737.6 KB


/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_92159/3521151496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [95]:
y_pred = xgb_model.predict(data)

In [96]:
len(y_pred)

13485

In [97]:
predictions = pd.DataFrame(y_pred, columns=['price'])

In [98]:
predictions['id'] = range(0, len(predictions))
predictions

,price,id
0,2894.447998,0
1,5471.280273,1
2,9712.176758,2
3,4107.557617,3
4,1657.010010,4
...,...,...
13480,1654.638428,13480
13481,2351.010742,13481
13482,3225.466797,13482
13483,1978.620239,13483


In [99]:
predictions = predictions[['id', 'price']]
predictions

,id,price
0,0,2894.447998
1,1,5471.280273
2,2,9712.176758
3,3,4107.557617
4,4,1657.010010
...,...,...
13480,13480,1654.638428
13481,13481,2351.010742
13482,13482,3225.466797
13483,13483,1978.620239


In [100]:
predictions.to_csv('../data/submission/submission_XgBoost_MSR_409.csv', index = False)

In [ ]:
#cross validation y grid search